In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

In [2]:
test = fits.open("../data/NGC4030/mcmc/NGC4030-SQUARE0.6/BETA-CORR/NGC4030-SQUARE0.6-binid-0-700-samples-run-0.fits")

In [11]:
test[1].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =              3520108 / length of dimension 1                          
NAXIS2  =                  701 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                    4 / number of table fields                         
TTYPE1  = 'bin     '                                                            
TFORM1  = 'J       '                                                            
TTYPE2  = 'samples '                                                            
TFORM2  = '440000D '                                                            
TDIM2   = '(4,1100,100)'    